In [1]:
import pandas as pd
from pathlib import Path
import os

In [3]:
directory = Path(os.getcwd())   # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\HKUST-x-J.P.-Morgan-Fintech-Mentorship-Program\\model_testing\\supervised'
directory = directory.parent.parent.parent.joinpath("data")  # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\data'
df = pd.read_pickle(directory.joinpath("stocktwits_raw_sample.pkl"))
df

,id,text,time,sentiment
0,417739155,$GOEV it’s so easy to see insiders buying befo...,1639643526,
1,417739053,$QQQ $AAPL $SPY $NDX Santa’s rally has an addr...,1639643425,
2,417738922,$AAPL Futures Ripping Means \nAAPL up 2% to 4%...,1639643308,
3,417738486,$AAPL pullback today,1639642896,Bearish
4,417738230,$AAPL look! The teddy bear picnic is starting!...,1639642665,Bullish
...,...,...,...,...
47815,419624841,$PYPL $V holding up well to the long side. $MC...,1640145912,
47816,419623909,$V $MSFT $MMM $HPQ suggested for $XMI US ARCA ...,1640145541,
47817,419618889,$SQ and $V similar charts. We will see $180,1640143905,
47818,419605709,$V damn putting in some work last couple of we...,1640139998,Bullish


In [6]:
import re
# given a text, find the related symbols and remove it 
# return: a string of text and a list of symbols
def find_and_remove_syb(text):
    symbols = re.findall(r"\B[$][a-zA-Z]*", text)
    symbols = [i for i in symbols if i != "$"]
    for symbol in symbols:
        text = text.replace(symbol, "")
    return text, symbols

# create a new dataframe
df_stocks = pd.DataFrame(columns=['text', 'time', 'symbols'])
for ind in range(len(df)):
    text, symbol = find_and_remove_syb(df.iloc[ind].text)
    if len(symbol) == 1:
        df_stocks = df_stocks.append({"text": text, "symbols": symbol[0], "time": df.iloc[ind].time}, ignore_index=True)
df_stocks

,text,time,symbols
0,pullback today,1639642896,$AAPL
1,look! The teddy bear picnic is starting! Lol,1639642665,$AAPL
2,,1639642565,$AAPL
3,classic inflation of calls in order to load u...,1639641515,$AAPL
4,"Guys, AAPL consolidating well and ready to go...",1639641435,$AAPL
...,...,...,...
36276,great day 👍👍,1640153385,$V
36277,inverse head n shoulders.... 222 possible tom...,1640149580,$V
36278,[15s. delayed]: Issued Press Release on Decem...,1640148346,$V
36279,damn putting in some work last couple of weeks..,1640139998,$V


In [10]:
def remove_website(text):
    for word in text.split():
        if 'http' in word:
            text = text.replace(word, '')
    return text

df_stocks['text'] = [remove_website(i) for i in list(df_stocks['text'])]

import html
df_stocks['text'] = [html.unescape(i) for i in list(df_stocks['text'])]

df_stocks['symbols'] = [i.upper() for i in df_stocks['symbols']]

In [12]:
df_stocks['symbols'].value_counts()

$TSLA    16052
$AAPL     5588
$PYPL     4177
$FB       3566
$MSFT     2699
$BAC       879
$V         805
$JPM       714
$JD        621
$MS        505
$GS        383
$KO        292
Name: symbols, dtype: int64

In [13]:
df_stocks.to_pickle(directory.joinpath("stocktwits_processed_without_multiple_sample.pkl"))

In [21]:
directory = Path(os.getcwd())   # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\HKUST-x-J.P.-Morgan-Fintech-Mentorship-Program\\model_testing\\supervised'
directory = directory.parent.parent.parent.joinpath("data")  # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\data'
df = pd.read_pickle(directory.joinpath("stocktwits_processed_without_multiple_full.pkl"))
df = df.drop_duplicates(subset=['text'], keep='first')
df = df.reset_index(drop=True)
df

,id,text,time,sentiment,symbols
0,411887437,I'm buying 100 put contracts for the $160 Jan...,1638289746,Bearish,$AAPL
1,411887365,"yeah, yeah. We all own it…that’s the point.",1638289733,,$AAPL
2,411886866,,1638289630,,$AAPL
3,411886768,When every dummy and his mother is telling yo...,1638289610,Bearish,$AAPL
4,411886589,volume more than double average. That’s insan...,1638289575,Bullish,$AAPL
...,...,...,...,...,...
195422,420636897,$300 price target as Holiday retail sales shi...,1640505701,Bullish,$V
195423,420632925,"History shows that towards year end, markets ...",1640502969,,$V
195424,420612505,you think with these airline cancellation new...,1640468616,,$V
195425,420610594,SweepCast observed: with Unusual Options Acti...,1640466249,,$V


In [36]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
df['text'] = df['text'].apply(deEmojify)
df['text'].value_counts()

                                                                                                                                          417
.X                                                                                                                                        346
                                                                                                                                           72
.X                                                                                                                                         56
-X                                                                                                                                         28
                                                                                                                                         ... 
.X bears get lost now bye bye                                                                                                               1
.X Mar

In [40]:
def blank(text):
    if text != None and len(text.split()) == 0:
        return None
    return text

In [41]:
df['text'] = df['text'].apply(blank)
df

,id,text,time,sentiment,symbols
0,411887437,I'm buying 100 put contracts for the $160 Jan...,1638289746,Bearish,$AAPL
1,411887365,"yeah, yeah. We all own it…that’s the point.",1638289733,,$AAPL
2,411886866,None,1638289630,,$AAPL
3,411886768,When every dummy and his mother is telling yo...,1638289610,Bearish,$AAPL
4,411886589,volume more than double average. That’s insan...,1638289575,Bullish,$AAPL
...,...,...,...,...,...
195422,420636897,$300 price target as Holiday retail sales shi...,1640505701,Bullish,$V
195423,420632925,"History shows that towards year end, markets ...",1640502969,,$V
195424,420612505,you think with these airline cancellation new...,1640468616,,$V
195425,420610594,SweepCast observed: with Unusual Options Acti...,1640466249,,$V


In [45]:
df = df.drop_duplicates(subset=['text'], keep='first')
df = df.reset_index(drop=True)
df = df.dropna(subset=['text'])
df.to_pickle(directory.joinpath("stocktwits_processed_without_multiple_full.pkl"))

In [43]:
df['text'].value_counts()

 I'm buying 100 put contracts for the $160 Jan22  expiration. I don't have to hide my trade by selling $160 calls.                                                                                                  1
 whatever gained yesday lost today                                                                                                                                                                                  1
 3000 by Friday                                                                                                                                                                                                     1
 MOMMA THERE GOES THAT MAN‼️‼️‼️‼️\n\nMR GOOGLE HIMSELF \n\nMADE A KILLING BUYING PUTS UP TO THIS DAY, NOW TIME TO RIDE IT THE OTHER WAY\n\nMOMMA THERE HE GO‼️‼️‼️‼️\n\n @incognit0trader @Greenlambo @Golden09    1
 SPY is going to break ATHS tommorow and this may start following its direction, shares entry here for long term/short term                     